In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
from scipy import ndimage
import math
from scipy import signal

**AJUSTANDO O TAMANHO DA IMAGEM**

In [44]:
def acha_template(template,amostra):
    resized_imagem = cv2.resize(amostra,(1150,210),interpolation = cv2.INTER_LINEAR) 
    resized_template = cv2.resize(template,(1050,210),interpolation = cv2.INTER_LINEAR)
    mask = mascara_canhoto(resized_imagem) #CRIA A MASCARA 
    mask_template = mascara_template(resized_template)
    fator_shift,img_shift = translacao(mask_template,mask)
    max_tp = 0 #VARIAVEL DE INICIALIZAÇÃO PARA PEGAR O MAIOR VALOR DE CORRELAÇÃO
    fator_escala = 0 #FATOR DE CORREÇÃO DE ESCALA
    w, h = mask_template.shape[:2] 
    for ajuste_escala in np.linspace(0.88,1.1,30):
        novo_w = int(np.ceil(ajuste_escala*w))
        novo_h = int(np.ceil(ajuste_escala*h))
        resized_image = cv2.resize(img_shift,(novo_h,novo_w),interpolation = cv2.INTER_LINEAR)
        img_rotacionada,fator_de_rot = rotaciona(mask_template,resized_image)
        tp = cv2.matchTemplate(img_rotacionada,mask_template,cv2.TM_CCOEFF_NORMED)
        if tp.max() > max_tp:
            max_tp = tp.max()
            best_rescale = resized_image
            fator_escala = ajuste_escala
    return max_tp, fator_escala,fator_de_rot

**FAZENDO A TRANSLAÇÃO**

In [45]:
def translacao(template,imagem):
    num_rows, num_cols = imagem.shape[:2]
    max_tm = 0
    fator_shifting = 0
    best_translacao = imagem
    for fator_shift in range(0,num_cols):
        translation_matrix = np.float32([[1,0,fator_shift], [0,1,0]])
        matriz_translada = cv2.warpAffine(imagem, translation_matrix, (num_cols,num_rows))
        tm = cv2.matchTemplate(matriz_translada,template,cv2.TM_CCOEFF_NORMED)
        if tm.max() > max_tm:
            max_tm = tm.max()
            best_translacao = matriz_translada
            fator_shifting = fator_shift
    return fator_shifting,best_translacao

**REALIZA A ROTAÇÃO**

In [46]:
def rotaciona(template,amostra):
    max_tp = -3
    w, h = amostra.shape[:2]
    fator2 = 0
    best_image = amostra 
    centro = (int(np.ceil(w/2)),int(np.ceil(h/2)))
    for par_rot in np.linspace(-3,3,20):
        rot_img = cv2.getRotationMatrix2D(centro,par_rot,1.0)
        rotated = cv2.warpAffine(amostra, rot_img, (h, w))
        tp = cv2.matchTemplate(rotated,template,cv2.TM_CCOEFF_NORMED)
        if tp.max() > max_tp:
            max_tp = tp.max()
            best_image = rotated
            fator2 = par_rot
    return best_image, fator2

**CRIA MASCARA**

In [47]:
def mascara_canhoto(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_black_1 = np.array([0, 0, 0])
    upper_black_1 = np.array([200, 200, 210])
    mask = cv2.inRange(hsv, lower_black_1, upper_black_1)
    return mask

In [48]:
def mascara_template(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_black_1 = np.array([0, 0, 0])
    upper_black_1 = np.array([0, 0, 10])
    mask = cv2.inRange(hsv, lower_black_1, upper_black_1)
    return mask

**MOSTRA MAIOR CORRELAÇÃO**

In [49]:
def maior_correlacao(var2,var3,var4,var):
    if( (var2 > var3) and (var2>var4) ):
        print(f"template: {var},  maior correlação é do template 2")
    elif( (var3 > var2) and (var3 > var4) ):
        print(f"template: {var},  maior correlação é do template 3")
    elif( (var4 > var3) and (var4 > var2) ):
        print(f"template: {var},  maior correlação é do template 4")
    

In [50]:
for i in range(1,9):
    var = 2
    strin2 = "m2_00"+str(i)+".jpg"
    amostra   = cv2.imread(strin2)
    template2 = cv2.imread("m2.pgm")
    template3 = cv2.imread("m3.pgm")
    template4 = cv2.imread("m4.pgm")
    mask = mascara(amostra)
    corr2, escala2,ang2= acha_template(template2,amostra);
    corr3, escala3,ang3= acha_template(template3,amostra);
    corr4, escala4,ang4= acha_template(template4,amostra);
    maior_correlacao(corr2,corr3,corr4,var)
    print(f"corr2: {corr2}")
    print(f"corr3: {corr3}")
    print(f"corr4: {corr4}")
    #print(f"ampliação2: {escala2}")
    #print(f"ampliação3: {escala3}")
    #print(f"ampliação4: {escala4}")
    #print(f"angulo2: {ang2}")
    #print(f"angulo3: {ang3}")
    #print(f"angulo4: {ang4}")
for i in range(1,4):
    var =3
    strin3 = "m3_00"+str(i)+".jpg"
    amostra   = cv2.imread(strin3)
    template2 = cv2.imread("m2.pgm")
    template3 = cv2.imread("m3.pgm")
    template4 = cv2.imread("m4.pgm")
    mask = mascara(amostra)
    corr2, escala2,ang2= acha_template(template2,amostra);
    corr3, escala3,ang3= acha_template(template3,amostra);
    corr4, escala4,ang4= acha_template(template4,amostra);
    maior_correlacao(corr2,corr3,corr4,var)
    print(f"corr2: {corr2}")
    print(f"corr3: {corr3}")
    print(f"corr4: {corr4}")
    #print(f"ampliação2: {escala2}")
    #print(f"ampliação3: {escala3}")
    #print(f"ampliação4: {escala4}")
    #print(f"angulo2: {ang2}")
    #print(f"angulo3: {ang3}")
    #print(f"angulo4: {ang4}")
for i in range(1,9):
    var = 4
    strin4 = "m4_00"+str(i)+".jpg"
    amostra   = cv2.imread(strin4)
    template2 = cv2.imread("m2.pgm")
    template3 = cv2.imread("m3.pgm")
    template4 = cv2.imread("m4.pgm")
    mask = mascara(amostra)
    corr2, escala2,ang2= acha_template(template2,amostra);
    corr3, escala3,ang3= acha_template(template3,amostra);
    corr4, escala4,ang4= acha_template(template4,amostra);
    maior_correlacao(corr2,corr3,corr4,var)
    print(f"corr2: {corr2}")
    print(f"corr3: {corr3}")
    print(f"corr4: {corr4}")
    #print(f"ampliação2: {escala2}")
    #print(f"ampliação3: {escala3}")
    #print(f"ampliação4: {escala4}")
    #print(f"angulo2: {ang2}")
    #print(f"angulo3: {ang3}")
    #print(f"angulo4: {ang4}")

template: 2,  maior correlação é do template 2
corr2: 0.4112885296344757
corr3: 0.1689312756061554
corr4: 0.20860040187835693
template: 2,  maior correlação é do template 2
corr2: 0.4674333930015564
corr3: 0.2124308943748474
corr4: 0.21326085925102234
template: 2,  maior correlação é do template 2
corr2: 0.21362820267677307
corr3: 0.1518198549747467
corr4: 0.17322437465190887
template: 2,  maior correlação é do template 2
corr2: 0.3769210875034332
corr3: 0.258441686630249
corr4: 0.24722230434417725
template: 2,  maior correlação é do template 4
corr2: 0.11685974150896072
corr3: 0.1948191076517105
corr4: 0.23153777420520782
template: 2,  maior correlação é do template 4
corr2: 0.13917101919651031
corr3: 0.22020801901817322
corr4: 0.2390228807926178
template: 2,  maior correlação é do template 3
corr2: 0.21965010464191437
corr3: 0.26708847284317017
corr4: 0.2324843555688858
template: 2,  maior correlação é do template 2
corr2: 0.3989461660385132
corr3: 0.1325773000717163
corr4: 0.0449057